In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [16]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   80537 KB |  426891 KB |    1141 MB |    1063 MB |
|       from large pool |   74880 KB |  398592 KB |    1075 MB |    1002 MB |
|       from small pool |    5657 KB |   28620 KB |      66 MB |      60 MB |
|---------------------------------------------------------------------------|
| Active memory         |   80537 KB |  426891 KB |    1141 MB |    1063 MB |
|       from large pool |   74880 KB |  398592 KB |    1075 MB |

In [31]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (res):
                    R_backbone has       2294 params (~ 50.6) %
                    t_backbone has       2243 params (~ 49.4) %
total params:                 4537
cam2cam estimation => adding 4537 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@17.05.2021-10:21:40
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 5.934, TRANS ~ 66.047, POSE ~ 76279.547, ROTO ~ 7.796, 3D ~ 611.996, SELF R ~ 53.145, SELF t ~ 11.653, TOTAL ~ 781.672
#[cam2cam]: training batch iter 0 MPJPE: ~ 443.278 mm
#[cam2cam]: training batch iter 1 losses: GEO ~ 6.086, TRANS ~ 66.340, POSE ~ 73597.039, ROTO ~ 7.660, 3D ~ 591.075, SELF R ~ 51.965, SELF t ~ 11.827, TOTAL ~ 761.312
#[cam2cam]: training batch iter 1 MPJPE: ~ 414.192 mm
#[cam2cam]: training batch iter 2 losses: GEO ~ 6.228, TRANS ~ 67.038, POSE ~ 77413.648, ROTO ~ 7.170, 3D ~ 611.064, SELF R ~ 48.329, SELF t ~ 12.5